<a href="https://colab.research.google.com/github/Satorumi/Machine-Learning/blob/main/GANs_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GANs - Generative Adversarial Networks

There are two neural networks: a *Generator* and a *Discriminator*. The generator generates a "fake" sample given a random vector/matrix, and the discriminator attempts to detect whether a given sample is "real" (picked from the training data) or "fake" (generated by the generator). Training happens in tandem: we train the discriminator for a few epochs, then train the generator for a few epochs, and repeat. This way both the generator and the discriminator get better at doing their jobs. 

GANs however, can be notoriously difficult to train, and are extremely sensitive to hyperparameters, activation functions and regularization.
ref: [Generating Images using Generative Adversarial Networks](https://jovian.ai/aakashns/06b-anime-dcgan)

####Import Libraries

In [ ]:
!pip install opendatasets --upgrade --quiet

In [2]:
import os
import opendatasets as od

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torchvision.utils
import numpy as np
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

ModuleNotFoundError: ignored

####Import Dataset

In [ ]:
dataset_url =
# use opendatasets to import kaggle datset
od.download(dataset_url)

In [ ]:
data_dir = 

###Preprocessing Dataset
Load this dataset using the ImageFolder class from torchvision. Resize and normalize images' pixels value. Then, create a dataloader

Define device

In [ ]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [ ]:
#  define the image size which all image will have
image_size = 64
batch_size = 128
data_stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

Load and transform data

In [ ]:
transformations = transforms.Compose([transform=transforms.Compose([
                    # resize all image to the same size                                              
                    transforms.Resize(image_size),
                    transforms.CenterCrop(image_size),
                    transforms.ToTensor(), # convert to tensor obj
                    # normalize pixel with computed stats
                    transforms.Normalize(*data_stats)
                                    ])

In [ ]:
dataset = ImageFolder(data_dir, transforms=transformations)
# using torch.utils.data.DataLoader
dataloader = Dataloader(dataset, batch_size)

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)

# compute the standard deviation and mean og img
mean, std = 0., 0.
nb_samples = 0.

for data in train_dataloader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
stats = mean, std

Explore And Visualize Data

In [ ]:
# display a batch of img
batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
# denormalize and display img
plt.imshow(np.transpose(utils.make_grid(real_batch[0].to(device)[:64], 
                        padding=2, normalize=True).cpu(),(1,2,0)))

###Buid GANs Model

####Discriminator Model
The discriminator takes an image as input, and tries to classify it as "real" or "generated"

In [ ]:
# Define Discriminator model with Convolutional Network
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.classification = nn.Sequential(
        
        # input layer with 3 x 64 x 64
        nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False), # out 64
        nn.BatchNorm2d(64), # add a batch normalization 64
        # leaky relu level layer return very small output for negative value
        nn.LeakyReLU(0.2, inplace=True), # size: 64 x 32 x 32

        nn.Conv2d(64, 128, 4, 2, 1, False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True), # size: 128 x 16 x 16

        nn.Conv2d(128, 256, 4, 2, 1, False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace=True), # size: 256 x 8 x 8

        nn.Conv2d(256, 512, 4, 2, 1, False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace=True), # size: 512 x 4 x 4

        nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=2 False)
        nn.Flatten(),
        
        # use sigmoid activation func for binary classification
        nn.Sigmoid()
    )

  def forward(self, X):
    return self.classification(X)

In [ ]:
discriminator_model = Discriminator().to(device)

####Generator Model
The input to the generator is typically a vector or a matrix of random numbers (latent tensor) which is used as a seed for generating an image. The generator will convert a latent tensor of shape (128, 1, 1) into an image tensor of shape 3 x 28 x 28. To achive this, we'll use the ConvTranspose2d layer from PyTorch, which is performs to as a transposed convolution (or deconvolution). [Learn more](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md#transposed-convolution-animations)

In [ ]:
latent_size = 128

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    
    self.generate = nn.Sequential(
        # input size: latent x 1 x 1
        nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True) # size: 512 x 4 x 4

        nn.ConvTranspose2d(512, 256, 4, 2, 1, False),
        nn.BatchNorm2d(256),
        nn.ReLU(True), # size 256 x 8 x 8

        nn.ConvTranspose2d(256, 128, 4, 2, 1, False),
        nn.BatchNorm2d(128),
        nn.ReLU(True), # size 128 x 16 x 16

        nn.ConvTranspose2d(128, 64, 4, 2, 1, False),
        nn.BatchNorm2d(64),
        nn.ReLU(True), # size 64 x 32 x 32

        # output layer: size 3 x 64 x 64
        nn.ConvTranspose2d(64, 3, 4, 2, 1, False),
        nn.Tanh() # activation function range val from -1 -> 1
    )

  def forward(X):
    return self.generate(X)

In [ ]:
generator_model = Generator().to(device)

Test generator

In [ ]:
# use a fixed latent to see how the generated images 
# develop as we train the model
fixed_latent = torch.randn(batch_size, latent_size, 1, 1, device=device) # random latent tensors

generated_img = generator_model(test_x)
print(generated_img.shape)
plt.imshow(np.transpose(utils.make_grid(generated_img[0].to(device)[:64], 
                        padding=2, normalize=True).cpu(),(1,2,0)))

###Training Models

Define loss function and optimizer

In [ ]:
# define the loss function
loss_function = nn.BCELoss()

d_optimizer = torch.optim.Adam(discriminator_model.parameters(), learning_rate)
g_optimizer = torch.optim.Adam(generator_model.parameters(), learning_rate)

# encoded binary classification
real_classified = 1 
generated_classified = 0 # represent generated img



####Steps involved in training the discriminator.

We expect the discriminator to output 1 if the image was picked from the real MNIST dataset, and 0 if it was generated using the generator network.

1. We first pass a batch of real images, and compute the loss, setting the target labels to 1.

2. Then we pass a batch of fake images (generated using the generator) pass them into the discriminator, and compute the loss, setting the target labels to 0.

3. Finally we add the two losses and use the overall loss to perform gradient descent to adjust the weights of the discriminator.

In [ ]:
def train_discriminator(images, discriminator_model, generator_model, d_optimizer, g_optimizer):
  
  real_classified = torch.ones(batch_size, 1).to(device) # 1 represent real img
  generated_classified = torch.zeros(batch_size, 0).to(device) # 0 represent generated img
  
  # passin real imgs in discrimininator model and get classfied results
  real_classification = discrimininator_model(images) 

  # compute the loss for real imgs
  real_loss = F.binary_cross_entropy(real_classification, real_classified)

  # generate a latent and img from generator
  z = torch.randn(batch_size, latent_size).to(device)
  generated_img = generator_model(z)
  # classified the generated img
  generator_classification = discriminator_model(generated_img)
  # compute the loss for generate imgs
  generate_loss = F.binary_cross_entropy(generator_classification, generated_classified)

  # combine losses -> overall loss
  loss = real_loss + generate_loss

  ## Backpropagation
  # reset gradients
  d_optimizer.no_grad()
  g_optimizer.no_grad()

  # compute gradient
  loss.backward()

  d_optimizer.step()

  return real_loss, generate_loss, loss.item()


use the binary cross entropy loss function to evaluate the loss of binary classification

####Steps invlove in Generator Training
1. We generate a batch of images using the generator, pass the into the discriminator.

2. We calculate the loss by setting the target labels to 1 i.e. real. We do this because the generator's objective is to "fool" the discriminator.

3. We use the loss to perform gradient descent i.e. change the weights of the generator, so it gets better at generating real-like images to "fool" the discriminator.

In [ ]:
def train_generator(discriminator_model, generator_model, loss_function, d_optimizer, g_optimizer):
  real_classified = torch.ones(batch_size, 1).to(device) # 1 represent real img

  z = torch.randn(batch_size, latent_size).to(device) # a random lantent
  # generated based on z
  generated_img = generator_model(z)
  # classified the generated img
  generator_classification = discriminator_model(generated_img)
  # compare with real img to get the loss as we want the fake to be classified aas real
  generator_loss = loss_function(generator_classification, real_classified)

  ## Backpropagation
  # reset gradients
  d_optimizer.no_grad()
  g_optimizer.no_grad()

  # compute gradient
  generator_loss.backward()

  g_optimizer.step()

  return generator_loss.item(), generated_img

####Training Process

In [ ]:
learning_rate = 1e-2
batch_size = 128
img_size = 64
epochs = 12

In [ ]:
num_batches = len(dataloader)
data_size = len(dataloader.dataset)

In [ ]:
# tracking the process and storing value
generated_imgs = []
generator_losses = []
discriminator_losses = []
real_scores = []
fake_scores = []

In [ ]:
for epoch in range(epochs):
  print(f'Epoch {epoch+1}\n-------------------------------')
  total_gloss, total_dloss = 0., 0.
  total_realscores, total_fakescores = 0., 0.

  for batch, (images, _ ) in enumerate(data_loader):
    # reshape all the images in a batch
    images = images.reshape(batch_size, -1).to(device)
    
    # train the discrimninator with batch of imgs
    real_score, fake_score, d_loss = train_discriminator(images, discriminator_model, generator_model, 
                                    loss_function, d_optimizer, g_optimizer)
    
    # train the generator
    generator_loss, generated_img = train_generator(discriminator_model, generator_model, 
                    loss_function, d_optimizer, g_optimizer)
    
    # print result for every 500 min-batches
    if batch % 500 == 0:
      cur_batch = (batch+1) * len(images)
      print(f"Discriminator Loss: {d_loss.item():>4f}, Generator Loss: {generator_loss.item():>4f}  [{cur_batch:>5d}/{data_size:>5d}]")
      
  
  # compute the mean stats for each epoch
  generator_losses.append((total_gloss/num_batches).item())
  discriminator_losses.append((total_dloss/num_batches).item())
  real_scores.append((total_realscores/num_batches).item())
  fake_scores.append((total_fakescores/num_batches).item())

  generated_imgs.append(generated_img) # save a sample of generator img
  

    

#### Full Training Implementation

In [ ]:
def fit(epochs, lr, dataloader):
  torch.cuda.empty_cache()

  generated_imgs = []
  generator_losses = []
  discriminator_losses = []
  real_scores = []
  fake_scores = []

  # define optimizer
  d_optimizer = torch.optim.Adam(discriminator_model.parameters(), learning_rate)
  g_optimizer = torch.optim.Adam(generator_model.parameters(), learning_rate)

  ## training process
  for epoch in range(epochs):
    print(f'Epoch {epoch+1}\n-------------------------------')

  total_gloss, total_dloss = 0., 0.
  total_realscores, total_fakescores = 0., 0.

  for batch, (images, _ ) in enumerate(data_loader):
    # reshape all the images in a batch
    images = images.reshape(batch_size, -1).to(device)
    
    # train the discrimninator and generator model
    real_score, fake_score, d_loss = train_discriminator(images, discriminator_model, generator_model, 
                                    loss_function, d_optimizer, g_optimizer):
    generator_loss, generated_img = train_generator(discriminator_model, generator_model, 
                    loss_function, d_optimizer, g_optimizer)
    
    # print result for every 500 min-batches
    if batch % 500 == 0:
      cur_batch = (batch+1) * len(images)
      print(f"Discriminator Loss: {d_loss.item():>4f}, Generator Loss: {generator_loss.item():>4f}  [{cur_batch:>5d}/{data_size:>5d}]")
      
     
  
  # compute the mean stats for each epoch
  generator_losses.append((total_gloss/num_batches).item())
  discriminator_losses.append((total_dloss/num_batches).item())
  real_scores.append((total_realscores/num_batches).item())
  fake_scores.append((total_fakescores/num_batches).item())

  generated_imgs.append(generated_img) # save a sample of generator img

  return generator_losses, discriminator_losses, real_scores, fake_scores, generated_imgs


In [ ]:
history = fit(epochs, learning_rate)

###Visualize result

####Losses and Scores

In [ ]:
generator_losses, discriminator_losses, real_scores, fake_scores, generated_imgs = history

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(range(epochs), generator_losses, 'r-x')
plt.plot(range(epochs), generator_losses, 'r-x')
plt.legend(['Discriminator', 'Generator'], loc='upper right')
plt.title('Discriminator and Genrator Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss Rate')
ply.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(range(epochs), real_scores, 'r-x')
plt.plot(range(epochs), fake_scores, 'r-x')
plt.legend(['Real Classification', 'Generated Classification'], loc='upper right')
plt.title('Discriminator Classification Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss Rate')
ply.show()

####Images

Genrated Process

In [ ]:
# display a video of generated imgs
fig = plt.figure(figsize=(10,10))
plt.axis('off')
# using matplotlib.animation for denormalized images
images = [[plt.imshow(np.tranpose(img, (1,2,0)), animated=True)] for img in generated_imgs]
animated = animation.ArtistAnimation(fig, images, interval=1000, repeat_delay=1000, blit=True)

HTML(animated.to_jshtml())

Final Results and Comparision

In [ ]:
plt.figure(figsize=(12,12))

# display real images
batch, (images, _ ) = next(iter(dataloader)) # get a batch from dataloader
# denormalize the first 64 imgs
images = torchvision.utils.make_grid(images).to(device)[:64], padding=5, normalize=True)
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title('Real Images')
plt.imshow(np.tranpose(images, (1,2,0))) 

# display generated images
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title('Generated Images')
# display the last img from generated list
plt.imshow(np.tranpose(generated_imgs[-1], (1,2,0)) # de normalize
plt.show()

### Save Checkpoints

In [ ]:
torch.save(generator_model.state_dict(), 'generator_model.pth')
torch.save(discriminator_model.state_dict(), 'discriminator_model.pth')